In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import random

In [3]:
import os
os.chdir('/Users/jc2822/Desktop')

In [4]:
df = pd.read_csv('frequency_EHE.csv')

In [7]:
df.shape

(46635, 11)

In [8]:
df.head()

,GEOID,NAME,STATE_NAME,year_numerical,event_count,avg_total_area_sq_mile,mean_event_count,sd,outlier_upper,outlier_lower,event_count_outlier_rm
0,1001,Autauga,Alabama,2008,3,617.968768,6.133333,4.51769,19.686404,-7.419737,3
1,1001,Autauga,Alabama,2009,8,617.968768,6.133333,4.51769,19.686404,-7.419737,8
2,1001,Autauga,Alabama,2010,14,617.968768,6.133333,4.51769,19.686404,-7.419737,14
3,1001,Autauga,Alabama,2011,16,617.968768,6.133333,4.51769,19.686404,-7.419737,16
4,1001,Autauga,Alabama,2012,6,617.968768,6.133333,4.51769,19.686404,-7.419737,6


In [7]:
def predict_event_count_for_county(data, GEOID, future_years=np.arange(2023,2037)):
    # Filter data for the current county
    county_data = data[data['GEOID'] == GEOID].sort_values('year_numerical').reset_index(drop=True)
    
    X = county_data['year_numerical'].values.reshape(-1, 1)
    y = county_data['event_count'].values.reshape(-1, 1)

    scaler_x = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler_x.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    def create_dataset(X, y, time_step=1):
        Xs, ys = [], []
        for i in range(len(X) - time_step):
            v = X[i:(i + time_step), 0]
            Xs.append(v)
            ys.append(y[i + time_step, 0])
        return np.array(Xs), np.array(ys)

    time_step = 1
    Xs, ys = create_dataset(X_scaled, y_scaled, time_step)
    Xs = np.reshape(Xs, (Xs.shape[0], Xs.shape[1], 1))

    X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, batch_size=1, epochs=100, verbose=0)

    y_test_pred_scaled = model.predict(X_test)
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)
    y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1))
    rmse = sqrt(mean_squared_error(y_test_actual, y_test_pred))
    errors = y_test_actual - y_test_pred
    error_std = np.std(errors)

    # Predict future event counts
    future_years_scaled = scaler_x.transform(future_years.reshape(-1, 1))
    future_years_scaled = np.reshape(future_years_scaled, (future_years_scaled.shape[0], time_step, 1))
    predicted_event_count_scaled = model.predict(future_years_scaled)
    predicted_event_count = scaler_y.inverse_transform(predicted_event_count_scaled)

    # Calculate confidence intervals based on error_std
    ci_multiplier = 1.96  # Approximation for 95% CI, assuming normal distribution of errors
    ci_low = predicted_event_count - ci_multiplier * error_std
    ci_high = predicted_event_count + ci_multiplier * error_std

    # Create output table with CI_low and CI_high
    output_df = pd.DataFrame({
        'GEOID': GEOID,
        'YEAR': future_years,
        'Predicted event count': predicted_event_count.flatten(),
        'CI_low': ci_low.flatten(),
        'CI_high': ci_high.flatten(),
        'RMSE': rmse
    })

    return output_df

In [52]:
final_results_df.to_csv('lstm_pred_freq.csv', index=False)

In [53]:
final_results_df.head()

,GEOID,YEAR,Predicted event count,RMSE
0,1001,2025,5.177353,6.138619
1,1001,2035,5.788127,6.138619
2,1003,2025,7.062937,3.922006
3,1003,2035,7.866003,3.922006
4,1005,2025,5.715767,6.428829


In [9]:
def predict_event_count_for_county(data, GEOID, future_years=np.arange(2023,2038)):
    # Filter data for the current county
    county_data = data[data['GEOID'] == GEOID].sort_values('year_numerical').reset_index(drop=True)
    
    X = county_data['year_numerical'].values.reshape(-1, 1)
    y = county_data['event_count'].values.reshape(-1, 1)

    scaler_x = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler_x.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    def create_dataset(X, y, time_step=1):
        Xs, ys = [], []
        for i in range(len(X) - time_step):
            v = X[i:(i + time_step), 0]
            Xs.append(v)
            ys.append(y[i + time_step, 0])
        return np.array(Xs), np.array(ys)

    time_step = 1
    Xs, ys = create_dataset(X_scaled, y_scaled, time_step)
    Xs = np.reshape(Xs, (Xs.shape[0], Xs.shape[1], 1))

    # Manually split the data into training and testing sets based on the last 5 years
    split_index = len(Xs) - 5  # Adjust based on your data's time frequency
    X_train, X_test = Xs[:split_index], Xs[split_index:]
    y_train, y_test = ys[:split_index], ys[split_index:]

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, batch_size=1, epochs=100, verbose=0)

    y_test_pred_scaled = model.predict(X_test)
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)
    y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1))
    rmse = sqrt(mean_squared_error(y_test_actual, y_test_pred))
    errors = y_test_actual - y_test_pred
    error_std = np.std(errors)

    # Predict future event counts
    future_years_scaled = scaler_x.transform(future_years.reshape(-1, 1))
    future_years_scaled = np.reshape(future_years_scaled, (future_years_scaled.shape[0], time_step, 1))
    predicted_event_count_scaled = model.predict(future_years_scaled)
    predicted_event_count = scaler_y.inverse_transform(predicted_event_count_scaled)

    # Calculate confidence intervals based on error_std
    ci_multiplier = 1.96  # Approximation for 95% CI, assuming normal distribution of errors
    ci_low = predicted_event_count - ci_multiplier * error_std
    ci_high = predicted_event_count + ci_multiplier * error_std

    # Create output table with CI_low and CI_high
    output_df = pd.DataFrame({
        'GEOID': GEOID,
        'YEAR': future_years,
        'Predicted event count': predicted_event_count.flatten(),
        'CI_low': ci_low.flatten(),
        'CI_high': ci_high.flatten(),
        'RMSE': rmse
    })

    return output_df


In [10]:
counties = df['GEOID'].unique()

results_list = []

for county_id in counties:
    result_df = predict_event_count_for_county(df, county_id)
    results_list.append(result_df)

final_results_df = pd.concat(results_list, ignore_index=True)

1/1 [==============================] - 0s 6ms/step


In [23]:
final_results_df.head()

,GEOID,YEAR,Predicted event count,CI_low,CI_high,RMSE
0,1001,2025,13.112583,11.081868,15.143298,4.084538
1,1001,2035,22.306513,20.275797,24.337229,4.084538
2,1003,2025,-4.290454,-18.966236,10.385326,12.476240
3,1003,2035,-7.916594,-22.592375,6.759188,12.476240
4,1005,2025,-6.973750,-16.485348,2.537848,11.234704


In [11]:
final_results_df.to_csv('lstm_pred_freq.csv', index=False)

In [12]:
def predict_by_county(data, GEOID, future_years=np.arange(2023,2038), test_set_selection='last', n_years=5):
    # Filter data for the current county
    county_data = data[data['GEOID'] == GEOID].sort_values('year_numerical').reset_index(drop=True)
    
    X = county_data['year_numerical'].values.reshape(-1, 1)
    y = county_data['event_count'].values.reshape(-1, 1)

    scaler_x = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler_x.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    def create_dataset(X, y, time_step=1):
        Xs, ys = [], []
        for i in range(len(X) - time_step):
            v = X[i:(i + time_step), 0]
            Xs.append(v)
            ys.append(y[i + time_step, 0])
        return np.array(Xs), np.array(ys)

    time_step = 1
    Xs, ys = create_dataset(X_scaled, y_scaled, time_step)
    Xs = np.reshape(Xs, (Xs.shape[0], Xs.shape[1], 1))

    # Different methods for splitting the dataset
    if test_set_selection == 'last':
        split_index = len(Xs) - n_years
        X_train, X_test = Xs[:split_index], Xs[split_index:]
        y_train, y_test = ys[:split_index], ys[split_index:]
    elif test_set_selection == 'first':
        split_index = n_years
        X_train, X_test = Xs[:split_index], Xs[split_index:]
        y_train, y_test = ys[:split_index], ys[split_index:]
    elif test_set_selection == 'random':
        indices = list(range(len(Xs)))
        random_indices = random.sample(indices, n_years)
        X_test = Xs[random_indices]
        y_test = ys[random_indices]
        X_train = np.delete(Xs, random_indices, axis=0)
        y_train = np.delete(ys, random_indices, axis=0)
    else:
        raise ValueError("Invalid test set selection method. Choose 'last', 'first', or 'random'.")

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, batch_size=1, epochs=100, verbose=0)

    # Predict on test set to calculate RMSE and standard deviation of errors
    y_test_pred_scaled = model.predict(X_test)
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)
    y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1))
    rmse = sqrt(mean_squared_error(y_test_actual, y_test_pred))
    errors = y_test_actual - y_test_pred
    error_std = np.std(errors)

    # Predict future event counts
    future_years_scaled = scaler_x.transform(future_years.reshape(-1, 1))
    future_years_scaled = np.reshape(future_years_scaled, (future_years_scaled.shape[0], time_step, 1))
    predicted_event_count_scaled = model.predict(future_years_scaled)
    predicted_event_count = scaler_y.inverse_transform(predicted_event_count_scaled)

    # Calculate confidence intervals based on error_std
    ci_multiplier = 1.96  # Approximation for 95% CI, assuming normal distribution of errors
    ci_low = predicted_event_count - ci_multiplier * error_std
    ci_high = predicted_event_count + ci_multiplier * error_std

    output_df = pd.DataFrame({
        'GEOID': GEOID,
        'YEAR': future_years,
        'Predicted event count': predicted_event_count.flatten(),
        'CI_low': ci_low.flatten(),
        'CI_high': ci_high.flatten(),
        'RMSE': rmse
    })

    return output_df

In [25]:
df['GEOID'].unique()

array([ 1001,  1003,  1005, ..., 56041, 56043, 56045])

In [ ]:
counties = df['GEOID'].unique()

results_list_first = []

for county_id in counties:
    result_df_first = predict_by_county(df, county_id,test_set_selection = 'last')
    results_list_first.append(result_df_first)

final_results_df_first = pd.concat(results_list_first, ignore_index=True)

1/1 [==============================] - 0s 6ms/step


In [28]:
final_results_df_first

,GEOID,YEAR,Predicted event count,CI_low,CI_high,RMSE
0,1001,2023,2.078803,-2.373478,6.531084,2.306758
1,1001,2024,1.587427,-2.864854,6.039708,2.306758
2,1001,2025,1.108807,-3.343473,5.561089,2.306758
3,1001,2026,0.643205,-3.809077,5.095486,2.306758
4,1001,2027,0.190824,-4.261457,4.643105,2.306758
5,1001,2028,-0.248177,-4.700458,4.204104,2.306758
6,1001,2029,-0.673691,-5.125972,3.778590,2.306758
7,1001,2030,-1.085659,-5.537940,3.366622,2.306758
8,1001,2031,-1.484068,-5.936349,2.968213,2.306758
9,1001,2032,-1.868948,-6.321229,2.583333,2.306758


In [ ]:
final_results_df_first.to_csv('lstm_pred_freq2.csv', index=False)